In [1]:
%matplotlib inline ## no need of plt.show()
%reload_ext autoreload
%autoreload 2

UsageError: unrecognized arguments: ## no need of plt.show()


In [2]:
# basics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import re, string, os, sys
from datetime import datetime
import warnings # To ignore any warnings (Good to have)
warnings.filterwarnings("ignore")

# viz
import seaborn as sns
from IPython.display import display, HTML, Image
from IPython.core.display import HTML 

# tree viz
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO  
import pydotplus

# machine learning
from sklearn.model_selection import train_test_split, ParameterGrid,KFold, StratifiedKFold
#from sklearn.cross_validation import KFold
from sklearn.linear_model import Ridge,Lasso
from sklearn.ensemble import RandomForestRegressor, forest, RandomForestClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import average_precision_score,precision_recall_curve
from math import sqrt
from sklearn.base import BaseEstimator
from sklearn import preprocessing # label encoding
from sklearn.metrics import roc_auc_score,roc_curve,classification_report,confusion_matrix,auc # auc
from sklearn.tree import DecisionTreeClassifier

## Data Summary

In [3]:
path="D:/case studies/wayfair/data/" # update this as path where you keep your data files while runnng notebook

In [150]:
missing_values = ["n/a", "na", "--","NA","NAN"] # checking NAs, NANs if present as string in dataframe 
train_df = pd.read_csv(path+"df_training_scholarjet.csv",na_values = missing_values)

test_df = pd.read_csv(path+"df_holdout_scholarjet.csv",na_values = missing_values)

In [151]:
test_save = test_df.copy()

In [117]:
train_df.drop(["Unnamed: 0","cuid"], axis =1,inplace=True)
test_df.drop(["Unnamed: 0","cuid"], axis =1,inplace=True)

In [ ]:
#df=test_df.copy()

# Feature Engineering

In [118]:
for df in [train_df, test_df]: 
    df["activeonsite_3days"] = (df.numvisitone>0) | (df.numvisitthreeone>0) 
    df["activeonsite_7days"] = (df.numvisitone>0) | (df.numvisitthreeone>0) | (df.numvisitseventhree>0)
    df["activeonsite_30days"] = (df.numvisitone>0) | (df.numvisitthreeone>0) | (df.numvisitseventhree>0) |  (df.numvisitthirtyseven>0)
    list_name = ["numorderone","numorderthreeone","numorderseventhree","numorderthirtyseven"]
    list_ord = list_name + ["numordersixtythirty","numorderyearsixty"]
    df["monthly_cycle"] = (df.loc[:,list_name].fillna(0).sum(axis=1)>0) & (df.numordersixtythirty>0) &(df.numorderyearsixty>0)
    df["orders_per_month"] = ((df.loc[:,list_ord].fillna(0).sum(axis=1))/(df.dayssinceenrollment))*30
    df["ordered_last_month"] = df.loc[:,list_name].fillna(0).sum(axis=1)>0 # recency of purchase
    df["ordered_last_3days"] = (df.numorderone>0) | (df.numorderthreeone>0) 
    df["ordered_last_7days"] = (df.numorderone>0) | (df.numorderthreeone>0) | (df.numorderseventhree>0)
    list_name_email = ["numemailsone","numemailsthreeone","numemailsseventhree","numemailsthirtyseven"]
    df["email_BAM_last_month"] = df.loc[:,list_name_email].fillna(0).sum(axis=1)>0
    df["email_BAM_last_3days"] = (df.numemailsone>0) | (df.numemailsthreeone>0) 
    df["email_BAM_last_7days"] = (df.numemailsone>0) | (df.numemailsthreeone>0)  | (df.numemailsseventhree>0)
    list_name_calls = ["numcallsone","numcallsthreeone","numcallsseventhree","numcallsthirtyseven"]
    df["calls_last_month"] = df.loc[:,list_name_calls].fillna(0).sum(axis=1)>0
    df["calls_last_3days"] = (df.numcallsone>0) | (df.numcallsthreeone>0) 
    df["calls_last_7days"] = (df.numcallsone>0) | (df.numcallsthreeone>0)  | (df.numcallsseventhree>0)
    list_em_call = list_name_email + list_name_calls 
    df["calls&emails_last_month"] = df.loc[:,list_em_call].fillna(0).sum(axis=1)>0
    list_atc = ["numatcone","numatcthreeone","numatcseventhree","numatcthirtyseven"]
    df["atc&no_purchase_last_month"] = (df.loc[:,list_name].fillna(0).sum(axis=1)==0) & (df.loc[:,list_atc].fillna(0).sum(axis=1)>0)
    df["atc_last_month"] = df.loc[:,list_atc].fillna(0).sum(axis=1)>0
    df["atc_last_3days"] = (df.numatcone>0) | (df.numatcthreeone>0) 
    df["atc_last_7days"] = (df.numatcone>0) | (df.numatcthreeone>0)  | (df.numatcseventhree>0)
    list_name_sku = ["numskusviewedone","numskusviewedthreeone","numskusviewedseventhree","numskusviewedthirtyseven"]
    df["sku_last_month"] = df.loc[:,list_name_sku].fillna(0).sum(axis=1)>0
    df["sku_last_3days"] = (df.numskusviewedone>0) | (df.numskusviewedthreeone>0) 
    df["sku_last_7days"] = (df.numskusviewedone>0) | (df.numskusviewedthreeone>0)  | (df.numskusviewedseventhree>0)
    list_name_search = [ "numsearchtermsone","numsearchtermsthreeone","numsearchtermsseventhree","numsearchtermsthirtyseven"]
    df["search_last_month"] = df.loc[:,list_name_search].fillna(0).sum(axis=1)>0
    df["search_last_3days"] = (df.numsearchtermsone>0) | (df.numsearchtermsthreeone>0) 
    df["search_last_7days"] = (df.numsearchtermsone>0) | (df.numsearchtermsthreeone>0)  | (df.numsearchtermsseventhree>0)
    list_name_tasks = ["numtasksfirstintroone","numtasksfirstintrothreeone","numtasksfirstintroseventhree","numtasksfirstintrothirtyseven"]
    df["tasks_last_month"] = df.loc[:,list_name_tasks].fillna(0).sum(axis=1)>0
    df["tasks_last_3days"] = (df.numtasksfirstintroone>0) | (df.numtasksfirstintrothreeone>0) 
    df["tasks_last_7days"] = (df.numtasksfirstintroone>0) | (df.numtasksfirstintrothreeone>0)  | (df.numtasksfirstintroseventhree>0)
    list_or_7 = ["numorderone","numorderthreeone","numorderseventhree"]
    list_act = ["numskusviewedone","numskusviewedthreeone","numskusviewedseventhree",
            "numsearchtermsone","numsearchtermsthreeone","numsearchtermsseventhree"]
    df["activity&no_purchase_7days"] = (df.loc[:,list_or_7].fillna(0).sum(axis=1)==0) & (df.loc[:,list_act].fillna(0).sum(axis=1)>0)


# Data Preprocessing

## Missing values

#### Adding is_na features
add a {name}_na column which specifies if the data was missing.

In [119]:
for d in [train_df, test_df]:  
    missing = d.isna().sum()
    cols_missing = list(missing[missing > 0].index)
    print(len(cols_missing))
    for name in cols_missing:
        d[name + '_na'] = pd.isnull(d[name])

168
167


Imputing missing values

In [120]:
for df in [train_df, test_df]: 
    df.fillna(value=-100,inplace=True) # rest we replace by a constant value, as missing data has its own meaning # 0 already present

### Boolean to Int

In [121]:
def bool_to_int(df):
    for col in df.columns:
        if df[col].dtype == 'bool':
            df[col] = df[col]*1

In [122]:
bool_to_int(train_df)
bool_to_int(test_df)

In [123]:
train_df.shape, test_df.shape

((28126, 379), (30375, 376))

## Categorical variables

### Mean target encoding - Regularized

In [124]:
def target_encoder_regularized(train, test, cols_encode, target, folds = 5):
    """
    Mean regularized target encoding based on kfold
    
    """
    
    kf = KFold(n_splits=folds, shuffle=True, random_state=1)
    #     set_trace()
    test_copy = test.reset_index().copy()
    train_copy = train.reset_index().copy()
    
    for col in cols_encode:      
        encoded_column = col + "_mean_target"
        train_copy[encoded_column] = np.zeros(len(train_copy))
        test_copy[encoded_column] = np.zeros(len(test_copy))
        
        for train_index, val_index in kf.split(train):

            mean_target =  train_copy.loc[train_index].groupby(col)[target].mean()

            train_copy.loc[val_index,encoded_column] = train_copy.loc[val_index, col].map(mean_target)
        
        global_mean = train_copy[target].mean()
        train_copy[encoded_column].fillna(global_mean, inplace=True)
        
        # making test encoding using full training data 
        test_copy[encoded_column] = test_copy[col].map(train_copy.groupby(col)[target].mean())
        test_copy[encoded_column].fillna(global_mean, inplace=True)
            
    return train_copy, test_copy

In [125]:
cat_columns = list(train_df.select_dtypes(include='object').columns) # 10 columns
cat_columns

['roll_up',
 'currentstatus',
 'companytypegroup',
 'team',
 'customersource',
 'accrole',
 'num_employees',
 'num_purchases_year',
 'cost_purchases_year',
 'enrollmentmethod']

In [126]:
df_train, df_test = target_encoder_regularized(train_df, test_df, cat_columns,target='convert_30', folds=5)

In [36]:
#df_train_v2.loc[df_train_v2.roll_up=="Unmanaged",["roll_up","roll_up_mean_target"]]
#df_val_v2.loc[:,["roll_up","roll_up_mean_target"]]

In [128]:
def drop_columns(df_train, df_test, cols):
    """
    Drop a list of columns from both train df and test df
    """
    
    df_train.drop(cols, inplace=True, axis=1)
    df_test.drop(cols, inplace=True, axis=1)

In [129]:
drop_columns(df_train, df_test, cat_columns+["index"])

In [130]:
df_train['orders_per_month'].replace(np.inf,-200,inplace=True)
df_test['orders_per_month'].replace(np.inf,-200,inplace=True)

In [131]:
df_test.shape, df_train.shape

((30375, 376), (28126, 379))

# Model building

In [50]:
X = df_train.copy().drop(['convert_30','revenue_30'], axis=1) # removing target variables
y = df_train.copy()['convert_30']

In [51]:
X.shape, y.shape, df_test.shape

((28126, 377), (28126,), (30375, 376))

In [52]:
imp_columns = ['numreturn',
 'numideaboardsixtythirty',
 'sumrevthreeone',
 'search_last_month',
 'sumrevseventhree',
 'avgpricesixtythirty_na',
 'numwims',
 'numcallsseventhree',
 'numtasksfirstintroseventhree',
 'numemailsseventhree',
 'percentresolved',
 'numcallsthreeone',
 'numorderthirtyseven',
 'numemailssixtythirty',
 'numcallssixtythirty',
 'percdirtythirty',
 'numinf',
 'numsearchtermsthreeone',
 'numordersixtythirty',
 'percemailclickedseventhree',
 'avgquoteprice',
 'numother',
 'percsecondsinbound',
 'numtasksfirstintrothirtyseven',
 'percemailclickedthreeone',
 'numcallsthirtyseven',
 'numtasksfirstintroyearsixty',
 'numloggedinthreeone',
 'numsearchtermsseventhree',
 'numvisitthreeone',
 'totalcalldurationseventhree',
 'sumrevsixtythirty',
 'numatcthirtyseven',
 'percemailopenedone',
 'numvisitone',
 'percemailopenedthreeone',
 'avgpriceone',
 'numsearchtermssixtythirty',
 'sumrevthirtyseven',
 'numcallsyearsixty',
 'numemaillist',
 'numsearchtermsone',
 'sumrevyearsixty',
 'numemailsyearsixty',
 'numloggedinseventhree',
 'numorderyearsixty',
 'numsecondsonsiteone',
 'totalcalldurationsixtythirty',
 'numideaboardyearsixty',
 'numideaboardthirtyseven',
 'numatcsixtythirty',
 'numselforder',
 'numsecondsonsitethreeone',
 'numskusviewedseventhree',
 'aov',
 'numskusviewedthreeone',
 'numtotalpageviewsone',
 'avgpriceseventhree_na',
 'numloggedinsixtythirty',
 'totalcalldurationthirtyseven',
 'avgpriceone_na',
 'dayssincelastord',
 'numatcyearsixty',
 'numskusviewedone',
 'percemailopenedseventhree',
 'numskusviewedsixtythirty',
 'avgpriceseventhree',
 'totalrev',
 'activeonsite_7days',
 'percemailclickedsixtythirty',
 'numsearchtermsthirtyseven',
 'totalcalldurationyearsixty',
 'percemailclickedthirtyseven',
 'companytypegroup_mean_target',
 'team_mean_target',
 'atc_last_month',
 'numvisitseventhree',
 'numvisitthirtyseven',
 'numloggedinthirtyseven',
 'currentapplicability',
 'percemailclickedyearsixty',
 'accrole_mean_target',
 'percemailopenedsixtythirty',
 'numsecondsonsitesixtythirty',
 'orders_per_month',
 'dayssincelastvisit',
 'numvisitsixtythirty',
 'percemailopenedyearsixty',
 'numsearchtermsyearsixty',
 'cost_purchases_year_mean_target',
 'numtotalpageviewssixtythirty',
 'num_purchases_year_mean_target',
 'avgatcprice',
 'roll_up_mean_target',
 'numloggedinyearsixty',
 'avgpricethirtyseven',
 'numvisityearsixty',
 'avgpricesixtythirty',
 'currentstatus_mean_target',
 'numsecondsonsiteyearsixty',
 'num_employees_mean_target',
 'sumatcprice',
 'percemailopenedthirtyseven',
 'avgpricethreeone',
 'numtotalpageviewsthreeone',
 'numtotalpageviewsyearsixty',
 'numvisittotal',
 'numskusviewedyearsixty',
 'enrollmentmethod_mean_target',
 'customersource_mean_target',
 'numsecondsonsitethirtyseven',
 'sku_last_3days',
 'dayssinceenrollment',
 'avgprice',
 'numskusviewedthirtyseven',
 'avgpriceyearsixty',
 'numsecondsonsiteseventhree',
 'numtotalpageviewsseventhree',
 'numtotalpageviewsthirtyseven']

In [53]:
X = X[imp_columns]
df_test = df_test[imp_columns] 

In [55]:
from sklearn.metrics import accuracy_score, log_loss
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier,RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, recall_score, precision_score


m = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                  colsample_bytree=0.4, gamma=0, learning_rate=0.1, max_delta_step=0,
                  max_depth=4, min_child_weight=3, missing=-999, n_estimators=100,
                  n_jobs=1, nthread=4, objective='binary:logistic', random_state=0,
                  reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=1337, silent=0,
                  subsample=0.75)
    

m.fit(X, y)  
prediction_score = m.predict_proba(df_test)[:,1]

In [57]:
df_test["prediction"] = prediction_score

In [59]:
df_test.to_csv("class_prediction_final.csv")

# Regression

In [133]:
#df_train_1 = df_train.loc[df_train.revenue_30>0]

In [154]:
#df_train_1.revenue_30

In [155]:
X = df_train.copy().drop(['convert_30','revenue_30'], axis=1) # removing target variables
y = df_train.copy()['revenue_30']

In [156]:
y.min(), y.max(),y.mean()

(0.0, 34986.6899, 75.03899719476578)

In [157]:
X.shape, y.shape, df_test.shape

((28126, 377), (28126,), (30375, 120))

In [158]:
imp_columns = ['numreturn',
 'numideaboardsixtythirty',
 'sumrevthreeone',
 'search_last_month',
 'sumrevseventhree',
 'avgpricesixtythirty_na',
 'numwims',
 'numcallsseventhree',
 'numtasksfirstintroseventhree',
 'numemailsseventhree',
 'percentresolved',
 'numcallsthreeone',
 'numorderthirtyseven',
 'numemailssixtythirty',
 'numcallssixtythirty',
 'percdirtythirty',
 'numinf',
 'numsearchtermsthreeone',
 'numordersixtythirty',
 'percemailclickedseventhree',
 'avgquoteprice',
 'numother',
 'percsecondsinbound',
 'numtasksfirstintrothirtyseven',
 'percemailclickedthreeone',
 'numcallsthirtyseven',
 'numtasksfirstintroyearsixty',
 'numloggedinthreeone',
 'numsearchtermsseventhree',
 'numvisitthreeone',
 'totalcalldurationseventhree',
 'sumrevsixtythirty',
 'numatcthirtyseven',
 'percemailopenedone',
 'numvisitone',
 'percemailopenedthreeone',
 'avgpriceone',
 'numsearchtermssixtythirty',
 'sumrevthirtyseven',
 'numcallsyearsixty',
 'numemaillist',
 'numsearchtermsone',
 'sumrevyearsixty',
 'numemailsyearsixty',
 'numloggedinseventhree',
 'numorderyearsixty',
 'numsecondsonsiteone',
 'totalcalldurationsixtythirty',
 'numideaboardyearsixty',
 'numideaboardthirtyseven',
 'numatcsixtythirty',
 'numselforder',
 'numsecondsonsitethreeone',
 'numskusviewedseventhree',
 'aov',
 'numskusviewedthreeone',
 'numtotalpageviewsone',
 'avgpriceseventhree_na',
 'numloggedinsixtythirty',
 'totalcalldurationthirtyseven',
 'avgpriceone_na',
 'dayssincelastord',
 'numatcyearsixty',
 'numskusviewedone',
 'percemailopenedseventhree',
 'numskusviewedsixtythirty',
 'avgpriceseventhree',
 'totalrev',
 'activeonsite_7days',
 'percemailclickedsixtythirty',
 'numsearchtermsthirtyseven',
 'totalcalldurationyearsixty',
 'percemailclickedthirtyseven',
 'companytypegroup_mean_target',
 'team_mean_target',
 'atc_last_month',
 'numvisitseventhree',
 'numvisitthirtyseven',
 'numloggedinthirtyseven',
 'currentapplicability',
 'percemailclickedyearsixty',
 'accrole_mean_target',
 'percemailopenedsixtythirty',
 'numsecondsonsitesixtythirty',
 'orders_per_month',
 'dayssincelastvisit',
 'numvisitsixtythirty',
 'percemailopenedyearsixty',
 'numsearchtermsyearsixty',
 'cost_purchases_year_mean_target',
 'numtotalpageviewssixtythirty',
 'num_purchases_year_mean_target',
 'avgatcprice',
 'roll_up_mean_target',
 'numloggedinyearsixty',
 'avgpricethirtyseven',
 'numvisityearsixty',
 'avgpricesixtythirty',
 'currentstatus_mean_target',
 'numsecondsonsiteyearsixty',
 'num_employees_mean_target',
 'sumatcprice',
 'percemailopenedthirtyseven',
 'avgpricethreeone',
 'numtotalpageviewsthreeone',
 'numtotalpageviewsyearsixty',
 'numvisittotal',
 'numskusviewedyearsixty',
 'enrollmentmethod_mean_target',
 'customersource_mean_target',
 'numsecondsonsitethirtyseven',
 'sku_last_3days',
 'dayssinceenrollment',
 'avgprice',
 'numskusviewedthirtyseven',
 'avgpriceyearsixty',
 'numsecondsonsiteseventhree',
 'numtotalpageviewsseventhree',
 'numtotalpageviewsthirtyseven']

In [159]:
X = X[imp_columns]
df_test = df_test[imp_columns] 

In [160]:
X.shape, y.shape, df_test.shape

((28126, 119), (28126,), (30375, 119))

In [ ]:
#y=np.log(y)

In [75]:
from sklearn.metrics import accuracy_score, log_loss
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier,RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from xgboost import XGBClassifier,XGBRegressor
from sklearn.metrics import f1_score, recall_score, precision_score


m = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                  colsample_bytree=0.4, gamma=0, learning_rate=0.1, max_delta_step=0,
                  max_depth=4, min_child_weight=3, missing=-999, n_estimators=100,
                  n_jobs=1, nthread=4, random_state=0,
                  reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=1337, silent=0,
                  subsample=0.75)
    

m.fit(X, y)  
prediction_score = m.predict(df_test)

In [161]:
m = XGBRegressor()
    
m.fit(X, y)  
prediction_score = m.predict(df_test)

In [162]:
prediction_score.min(), prediction_score.max(),prediction_score.mean()

(-171.66632, 10799.494, 65.44772)

In [163]:
df_test["rev_prediction"] = prediction_score

In [164]:
df_test.to_csv("reg_prediction_final.csv",index=False)